In [1]:
from bark import SAMPLE_RATE, generate_audio, preload_models
from scipy.io.wavfile import write as write_wav
from IPython.display import Audio

# download and load all models
preload_models()

# generate audio from text
text_prompt = """
     Hello, my name is Suno. And, uh — and I like pizza. [laughs] 
     But I also have other interests such as playing tic tac toe.
"""
audio_array = generate_audio(text_prompt)

# save audio to disk
write_wav("bark_generation.wav", SAMPLE_RATE, audio_array)
  
# play text in notebook
Audio(audio_array, rate=SAMPLE_RATE)

No GPU being used. Careful, inference might be very slow!


/Users/xuhuizhan/anaconda3/envs/py38/lib/python3.8/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
100%|██████████| 35/35 [01:57<00:00,  3.36s/it]


In [2]:
text_prompt = """
     ♪ In the jungle, the mighty jungle, the lion barks tonight ♪
"""
audio_array = generate_audio(text_prompt)

# save audio to disk
write_wav("bark_generation.wav", SAMPLE_RATE, audio_array)
  
# play text in notebook
Audio(audio_array, rate=SAMPLE_RATE)

100%|██████████| 35/35 [02:00<00:00,  3.44s/it]


## Record

In [29]:
import threading
import sounddevice as sd
from scipy.io.wavfile import write
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import time

fs = 44100  # Sample rate
channels = 1  # Mono recording
recording = np.array([])  # Placeholder for the recorded data
is_recording = False  # Flag to control recording

def record_audio(indata, frames, time, status):
    global recording
    recording = np.append(recording, indata.copy())

def update_recording_time():
    global is_recording
    start_time = time.time()
    while is_recording:
        elapsed_time = time.time() - start_time
        time_label.value = f"Recording... {elapsed_time:.2f} seconds"
        time.sleep(0.1)  # Update every 100ms

def start_recording(button):
    global is_recording, recording
    recording = np.array([])  # Reset recording
    is_recording = True
    print("Recording started...")
    # Start recording in a separate thread to avoid blocking
    threading.Thread(target=lambda: sd.InputStream(callback=record_audio, channels=channels, samplerate=fs).start()).start()
    # Update recording time in a separate thread
    threading.Thread(target=update_recording_time).start()

def stop_recording(button):
    global is_recording
    is_recording = False
    sd.stop()
    print("Recording stopped.")
    time_label.value = "Recording stopped."
    # Normalize to 16-bit range and save
    norm_audio = np.int16(recording / np.max(np.abs(recording)) * 32767)
    filename = 'speaker.wav'
    write(filename, fs, norm_audio)
    print(f"Audio saved as {filename}")

# Create buttons and label
start_button = widgets.Button(description="Start Recording")
stop_button = widgets.Button(description="Stop Recording")
time_label = widgets.Label(value="Press 'Start Recording' to begin")

# Bind the buttons
start_button.on_click(start_recording)
stop_button.on_click(stop_recording)

# Display widgets
display(start_button, stop_button, time_label)



Button(description='Start Recording', style=ButtonStyle())

Button(description='Stop Recording', style=ButtonStyle())

Label(value="Press 'Start Recording' to begin")

Recording started...
Recording stopped.
Audio saved as speaker.wav


In [33]:
from IPython.display import Audio
from TTS.api import TTS
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2", gpu=False)

# generate speech by cloning a voice using default settings
# tts.tts_to_file(text="Lucas's presenting issues include difficulties falling asleep, growing reluctance to attend school",
#                 file_path="output.wav",
#                 speaker_wav="speaker.wav",
#                 language="en")
wav = tts.tts(text="原谅太阳，原谅我", speaker_wav="speaker.wav", language="zh-cn")

Audio(wav, rate=22050)

 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.
 > Using model: xtts
 > Text splitted to sentences.
['原谅太阳，原谅我']
 > Processing time: 5.256278991699219
 > Real-time factor: 1.5343803188806366
